In [6]:
import os
import glob
import sys
import numpy as np
import cv2 as cv

In [7]:
hwd = os.getcwd()
cameras = glob.glob(os.path.join(hwd, 'E057_Cheeks_Puffed/*'))
frames = glob.glob(os.path.join(hwd, 'E057_Cheeks_Puffed/400002/*'))

In [8]:
cnt = 0
pairwise_flist = []
# for all pairs of cameras - 
for i in range(8):
    leftcamera = cameras[i]
    leftid = leftcamera.strip().split("/")[-1]
    for j in range(i+1, 8):
        rightcamera = cameras[j]
        rightid = rightcamera.strip().split("/")[-1]
        Fij = np.zeros((3,3))
        fmatches = -1
        # over all the frames of the video
        for k in range(len(frames)):
            
            frame_id = frames[k].strip().split("/")[-1]
            f1_name = os.path.join(leftcamera, frame_id)
            f2_name = os.path.join(rightcamera, frame_id)
            img1 = cv.imread(f1_name, cv.IMREAD_GRAYSCALE)
            img2 = cv.imread(f2_name, cv.IMREAD_GRAYSCALE)

            sift = cv.SIFT_create()
            # find the keypoints and descriptors with SIFT
            kp1, des1 = sift.detectAndCompute(img1, None)
            kp2, des2 = sift.detectAndCompute(img2, None)

            # FLANN parameters
            FLANN_INDEX_KDTREE = 1
            index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
            search_params = dict(checks=50)
            flann = cv.FlannBasedMatcher(index_params,search_params)
            matches = flann.knnMatch(des1,des2,k=2)
            
            if fmatches < len(list(matches)):
                # calculate F and update Fij
                pts1 = []
                pts2 = []
                # ratio test as per Lowe's paper
                for i,(m,n) in enumerate(matches):
                    if m.distance < 0.8*n.distance:
                        pts2.append(kp2[m.trainIdx].pt)
                        pts1.append(kp1[m.queryIdx].pt)
                        
                pts1 = np.int32(pts1)
                pts2 = np.int32(pts2)

                Fij, _ = cv.findFundamentalMat(pts1, pts2, cv.FM_LMEDS)
                fmatches = len(list(matches))
        pairwise_flist.append([leftid, rightid, Fij, fmatches])

In [9]:
len(pairwise_flist)

28

In [27]:
import pandas as pd
import numpy as np

krt = open("KRT_gaini", "r").readlines()
lines = len(krt)

cameras = dict()
images = dict()
trans = dict()
matrices = dict()
for i in range(0, lines):
    line = krt[i]
    line = line.strip()
    
    if line.startswith("4"):
        #intrinsics
        cam = line.strip() 
        fx = krt[i+1].split(" ")[0].strip()
        fy = krt[i+2].split(" ")[1].strip()
        cx = krt[i+1].split(" ")[2].strip()
        cy = krt[i+2].split(" ")[2].strip()
        cameras[cam] = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
           
        #extrinsics - extrinsic matrix to quaternions
        matrix = np.zeros((3,3), dtype=float)
        matrix[0,:] = krt[i+5].split(" ")[0:3]
        matrix[1,:] = krt[i+6].split(" ")[0:3]
        matrix[2,:] = krt[i+7].split(" ")[0:3]
        tx = krt[i+5].split(" ")[3].strip()
        ty = krt[i+6].split(" ")[3].strip()
        tz = krt[i+7].split(" ")[3].strip()
        
        T = np.array([float(tx), float(ty), float(tz)]).reshape((3,1))
        matrices[cam] = matrix        
        trans[cam] = T
    else:
        continue

In [17]:
im = dict()
cnt = 0
for i, pair in enumerate(pairwise_flist):
    if pair[0] not in im:
        im[pair[0]] = cnt
        cnt += 1

for i, pair in enumerate(pairwise_flist):
    if pair[1] not in im:
        im[pair[1]] = cnt
        cnt += 1

In [52]:
# Task 2 - loop through all the available pairs and concatenate the equations 
def get_initial_intrinsics(vars):
    err = 0
    for _, pair in enumerate(pairwise_flist):
        im1_id = pair[0]
        im2_id = pair[1]

        # intrinsics ground truth values       
        k1 = vars[3*(im[im1_id]): 3+3*(im[im1_id])]
        #a1 = np.array(cameras[im1_id], dtype=float)
        a1 = np.array([[7500*k1[0], 0, 1334*0.5*k1[1]], [0, 7800*k1[0], 2048*0.5*k1[2]], [0, 0, 1]])
        
        k2 = vars[3*(im[im2_id]): 3+3*(im[im2_id])]
        #a2 = np.array(cameras[im2_id], dtype=float)
        a2 = np.array([[7500*k2[0], 0, 1334*0.5*k2[1]], [0, 7800*k2[0], 2048*0.5*k2[2]], [0, 0, 1]])
        
        Fij = pair[2]
        r1, t1 = matrices[im1_id], trans[im1_id]
        r2, t2 = matrices[im2_id], trans[im2_id]
        
        r_rel = r2 @ r1.T
        t_rel = t2 - r_rel @ t1
        
        T1, T2, T3 = t_rel[0,0], t_rel[1,0], t_rel[2,0]
        t_cross = np.array(([0, -T3, T2], [T3, 0, -T1], [-T2, T1, 0]))
        E = t_cross @ r_rel
        
        eff = a2.T @ Fij @ a1
        lmd = np.sum(eff * E) / np.sum(eff * eff)
        delta = eff*lmd - E
        err += np.sum(delta ** 2) 
    
    return err

In [53]:
from scipy.optimize import minimize

vars0 = np.ones(8*3) #np.array([7000, 1334//2, 7000, 2048//2]*39)
res = minimize(get_initial_intrinsics, vars0)

In [54]:
print(res)

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 53612962.79575095
        x: [ 2.627e-02  1.894e+00 ... -5.865e-01  2.740e+00]
      nit: 208
      jac: [-1.140e+03 -4.000e+00 ...  3.500e+00  1.600e+01]
 hess_inv: [[ 2.768e-10  1.377e-09 ... -2.675e-09  4.104e-10]
            [ 1.377e-09  4.173e-08 ... -1.130e-09 -4.646e-09]
            ...
            [-2.675e-09 -1.130e-09 ...  1.769e-06 -6.723e-07]
            [ 4.104e-10 -4.646e-09 ... -6.723e-07  7.372e-07]]
     nfev: 7525
     njev: 301


In [ ]:
for i in range(0, len(res['x']), 3):
    k = res['x'][i:i+3]
    a1 = np.array([[7500*k[0], 0, 1334*0.5*k[1]], [0, 7800*k[0], 2048*0.5*k[2]], [0, 0, 1]])
    print(a1)